<a href="https://colab.research.google.com/github/Alina-Maistrenko/crypto-price-tracker/blob/main/notebook/crypto_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install pycoingecko pandas

In [12]:
from datetime import datetime, timedelta
import pandas as pd
from pycoingecko import CoinGeckoAPI

# 1. API Initialization
First, import the CoinGecko API client and create an instance to start fetching cryptocurrency data.

In [13]:
cg = CoinGeckoAPI()

# 2. Coins to Collect
We will collect price data for the following cryptocurrencies:

In [14]:
coins = {
    'bitcoin': 'BTC',
    'ethereum': 'ETH',
    'binancecoin': 'BNB',
    'cardano': 'ADA',
    'solana': 'SOL'
}

# 3. Function to Generate Date Ranges in 365-Day Intervals
This function yields start and end dates in chunks of 365 days between the given start and end dates.

In [15]:
def daterange(start_date, end_date, delta_days=365):
    current_start = start_date
    while current_start < end_date:
        current_end = min(current_start + timedelta(days=delta_days), end_date)
        yield current_start, current_end
        current_start = current_end + timedelta(days=1)

# 4. Define the Data Collection Period

In [16]:
start_date = datetime(2021, 1, 1)
end_date = datetime.now()

# 5. Initialize an Empty DataFrame
Create an empty pandas DataFrame to store combined price data for all cryptocurrencies.

In [17]:
combined_data = pd.DataFrame()

# 6. Main Data Collection Loop
This loop iterates over each cryptocurrency and fetches historical price data in defined date ranges, then appends it to a combined DataFrame.

In [18]:
for coin_id, symbol in coins.items():
    print(f"Processing {symbol}")
    for start, end in daterange(start_date, end_date):
        print(f"Loading data from {start.date()} to {end.date()}")
        from_ts = int(start.timestamp())
        to_ts = int(end.timestamp())
        try:
            data = cg.get_coin_market_chart_range_by_id(
                id=coin_id,
                vs_currency='usd',
                from_timestamp=from_ts,
                to_timestamp=to_ts
            )
            prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
            prices['date'] = pd.to_datetime(prices['timestamp'], unit='ms')
            prices['symbol'] = symbol
            prices = prices[['date', 'symbol', 'price']]
            combined_data = pd.concat([combined_data, prices], ignore_index=True)
        except Exception as e:
            print(f"Error loading {symbol} data from {start.date()} to {end.date()}: {e}")

Processing BTC
Loading data from 2021-01-01 to 2022-01-01
Error loading BTC data from 2021-01-01 to 2022-01-01: {'error': {'status': {'timestamp': '2025-07-07T23:04:38.551+00:00', 'error_code': 10012, 'error_message': 'Your request exceeds the allowed time range. Public API users are limited to querying historical data within the past 365 days. Upgrade to a paid plan to enjoy full historical data access: https://www.coingecko.com/en/api/pricing. '}}}
Loading data from 2022-01-02 to 2023-01-02
Error loading BTC data from 2022-01-02 to 2023-01-02: {'error': {'status': {'timestamp': '2025-07-07T23:04:38.652+00:00', 'error_code': 10012, 'error_message': 'Your request exceeds the allowed time range. Public API users are limited to querying historical data within the past 365 days. Upgrade to a paid plan to enjoy full historical data access: https://www.coingecko.com/en/api/pricing. '}}}
Loading data from 2023-01-03 to 2024-01-03
Error loading BTC data from 2023-01-03 to 2024-01-03: {'error'

# 7. Remove Duplicates
Remove duplicate entries based on date and symbol columns to ensure data integrity.

In [19]:
combined_data = combined_data.drop_duplicates(subset=['date', 'symbol']).reset_index(drop=True)


Notify that data collection has finished successfully


In [20]:
print("Data collection complete!")

Data collection complete!


# 8. Connect to (or create) SQLite database file in the data folder

In [21]:
import sqlite3

In [22]:
conn = sqlite3.connect('crypto_data.sqlite')

# 9. Save the combined DataFrame to a table named 'crypto_prices'

In [23]:
combined_data.to_sql('crypto_prices', conn, if_exists='replace', index=False)


925

# 10. Close the connection

In [24]:
conn.close()
print("Database saved to data/crypto_data.sqlite")

Database saved to data/crypto_data.sqlite


# 11. Download the Database from Colab to Your Computer


In [25]:
from google.colab import files
files.download('crypto_data.sqlite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>